### 1. (numpy) list 转成 array 实现减法
两个list不能直接做减法，需要转成array，计算后，再转回list

In [1]:
import numpy as np

channel_origin = [64, 64, 128, 128, 256, 256, 256, 256, 512, 512, 512, 512, 512, 512, 512, 512]  # vgg19 model
channel_prune = [35, 64, 128, 128, 255, 251, 222, 180, 113, 46, 44, 34, 17, 21, 33, 80]  # prune.txt

channel_variation = np.array(channel_origin) - np.array(channel_prune)
channel_variation = list(channel_variation)

labels = ["{:>3d}-{:>3d}={:>3d}".format(origin_item, channel_prune[i], channel_variation[i])
          for i, origin_item in enumerate(channel_origin)]
labels

[' 64- 35= 29',
 ' 64- 64=  0',
 '128-128=  0',
 '128-128=  0',
 '256-255=  1',
 '256-251=  5',
 '256-222= 34',
 '256-180= 76',
 '512-113=399',
 '512- 46=466',
 '512- 44=468',
 '512- 34=478',
 '512- 17=495',
 '512- 21=491',
 '512- 33=479',
 '512- 80=432']

### 2. np.reshape() 与 np.resize() 是不同的

- reshape 只能改变形状，不能改变原始输入包含的元素个数。
- resize 可以改变尺寸，只是进行简单的裁剪和填充。如果要更加精确的使用插值，则该函数不能实现。

In [2]:
import numpy as np

x = np.random.randn(4)
x, x.shape, x.size

(array([ 0.57080242,  0.56944377,  1.78976569, -0.84940764]), (4,), 4)

#### 2.1 reshape

In [8]:
y = x.reshape(4, 1)
y, y.shape, y.size

(array([[ 0.57080242],
        [ 0.56944377],
        [ 1.78976569],
        [-0.84940764]]),
 (4, 1),
 4)

#### 2.2 resize

In [9]:
z = np.resize(x, (4, 2))
z, z.shape, z.size

(array([[ 0.57080242,  0.56944377],
        [ 1.78976569, -0.84940764],
        [ 0.57080242,  0.56944377],
        [ 1.78976569, -0.84940764]]),
 (4, 2),
 8)

#### 2.3 `size` 与 `shape` 是不一样的

In [11]:
o = np.resize(z, (1,))
o, o.shape, o.size

(array([0.57080242]), (1,), 1)

In [12]:
m = np.random.randn(1, 1)
o = np.squeeze(m)
n = np.resize(o, (1,))
m, m.shape, o, o.shape, n, n.shape

(array([[0.42603275]]),
 (1, 1),
 array(0.42603275),
 (),
 array([0.42603275]),
 (1,))

### 3. np.argwhere(x > 0)  返回x中大于0的数组元组的索引

In [4]:
import torch
import numpy as np

x = [1, 2, 3, -1, -2]
x = torch.tensor(x)
y = np.argwhere(x.data.cpu().numpy() > 0)
y, y.shape

(array([[0],
        [1],
        [2]], dtype=int64),
 (3, 1))

In [5]:
z = np.squeeze(y)
z, z.shape

(array([0, 1, 2], dtype=int64), (3,))

### 4. numpy的排序 
    `np.sort()`返回从小到大排序后的序列
    `np.argsort()`返回从小到大排序后的索引序列位置

In [21]:
import numpy as np

arr = [1, 3, 5, 2, 4, 6]
y = np.sort(arr)   # data
i = np.argsort(arr)  # index
y, i

(array([1, 2, 3, 4, 5, 6]), array([0, 3, 1, 4, 2, 5], dtype=int64))

#### 4.1 numpy的比较 

In [26]:
mask = y > 4

mask, mask.astype(np.int), mask.astype(np.float)

(array([False, False, False, False,  True,  True]),
 array([0, 0, 0, 0, 1, 1]),
 array([0., 0., 0., 0., 1., 1.]))

#### 4.2 numpy的求和 

In [28]:
np.sum(mask), np.sum( mask.astype(np.int))

(2, 2)

### 5. 字符串空格占位
    ljust(width, [fillchar]) 输出width个字符，S左对齐，不足部分用fillchar填充，默认的为空格。 
    rjust(width, [fillchar])
    center(width, [fillchar])
    zfill(width) 把S变成width长，并在右对齐，不足部分用0补足。

In [1]:
string = 'How it is worked?'
lstr = string.ljust(50)
rstr = string.rjust(50)
cstr = string.center(50)
zstr = string.zfill(50)

lstr, rstr, cstr, zstr

('How it is worked?                                 ',
 '                                 How it is worked?',
 '                How it is worked?                 ',
 '000000000000000000000000000000000How it is worked?')

In [4]:
import torchnet as tnt

meter_loss = tnt.meter.AverageValueMeter()
meters_at = [tnt.meter.AverageValueMeter() for i in range(3)]
meter_loss, meters_at

(<torchnet.meter.averagevaluemeter.AverageValueMeter at 0x2481453c5f8>,
  <torchnet.meter.averagevaluemeter.AverageValueMeter at 0x2481453c6d8>])

In [6]:
vars(meter_loss)

{'n': 0,
 'sum': 0.0,
 'var': 0.0,
 'val': 0,
 'mean': nan,
 'mean_old': 0.0,
 'm_s': 0.0,
 'std': nan}

### 6. 数字转字节单位的字符串 

In [6]:
def bytes_to_human(n):
    symbols = ('K', 'M', 'G', 'T', 'P', 'E', 'Z', 'Y')
    prefix = {}
    for i, s in enumerate(symbols):
        prefix[s] = 1 << (i + 1) * 10
    for s in reversed(symbols):
        if n >= prefix[s]:
            value = float(n) / prefix[s]
            return '%.1f%s' % (value, s)
    return '%sB' % n

bytes_to_human(20000048)

'19.1M'

In [3]:
data_list = [38, 64, 'M', 127, 128, 'M', 254, 253, 220, 182, 'M', 117, 41, 40, 34, 'M', 18, 21, 33, 81]

string = '{}'.format(data_list).replace(',', '/')
string

"[38/ 64/ 'M'/ 127/ 128/ 'M'/ 254/ 253/ 220/ 182/ 'M'/ 117/ 41/ 40/ 34/ 'M'/ 18/ 21/ 33/ 81]"

### 7. load_model() 

In [5]:
import torch

filepath = r'G:\lizulin\network-slimming\logs\sparsity_resnet164_cifar10_s_1e_4\model_best.pth.tar'

pruned_model = torch.load(filepath)
pruned_model.keys()

dict_keys(['epoch', 'state_dict', 'best_prec1', 'optimizer'])

In [7]:
data_list = [23, 234, 23, 1, 98]
str(data_list).replace(',', '/')

'[23/ 234/ 23/ 1/ 98]'